In [47]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

In [53]:
class Spotify_recommendation(object):
    def __init__(self):
        self.df=pd.read_csv('top50.csv')
        self.numarical_df=None
        self.results=dict()

    def columns_renaming(self):
        self.df.columns=[i.lower().replace('.','') for i in self.df]
        
    def cleaning_str_columns(self):
        for i in ['trackname','artistname','genre']:
            self.df[i]=self.df[i].str.lower()
     
    def remove_column(self):
        self.df.drop(columns=['Unamed: 0'],axis=1,inplace=True)

    def normalization_algo(self, col):
        max_d=self.df[col].max()
        min_d=self.df[col].min()
        return (self.df[col]-min_d)/(max_d-min_d)
    
    def normalizing_col(self):
        df_numarical=self.df.select_dtypes(include=['int64'])
        self.numarical_df=df_numarical
        for i in self.numarical_df.columns:
            self.df[i]=self.normalization_algo(i)
    
    def cluster_creation(self):
        kn=KMeans(n_clusters=10)
        pred=kn.fit_predict(self.numarical_df)
        self.df['cluster_val']=pred
    
    def fit(self):
        total_songs =list(self.df['trackname'].unique())
        for i in total_songs:
            distance=[]
            song_res=self.df[self.df['trackname']==i].head(1).values[0]
            rem_data=self.df[self.df['trackname']!=i]
            
            for r_song in rem_data.values:
                dist=0
                for idx, col in enumerate(rem_data.columns):
                    if not col in ['trackname','artistname','genre']:
                        dist=dist + np.absolute(float(song_res[idx]-float(r_song[idx])))
                distance.append(dist)
            rem_data['distance']=distance
            rem_data=rem_data.sort_values('distance')
            temp=rem_data.to_dict(orient='records')
            
            self.results[i]=temp
    
    def predict(self, song_name,top_songs=5):
        return self.results[song_name][0:top_songs]
            

In [54]:
s=Spotify_recommendation()
s.columns_renaming()
s.cleaning_str_columns()
s.normalizing_col()
s.cluster_creation()
s.fit()

<ipython-input-53-7cc4cb40f598>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rem_data['distance']=distance


In [56]:
s.predict('bad guy')

[{'unnamed: 0': 0.4897959183673469,
  'trackname': 'bad guy (with justin bieber)',
  'artistname': 'billie eilish',
  'genre': 'electropop',
  'beatsperminute': 0.47619047619047616,
  'energy': 0.23214285714285715,
  'danceability': 0.6229508196721312,
  'loudnessdb': 0.0,
  'liveness': 0.1320754716981132,
  'valence': 0.6823529411764706,
  'length': 0.41237113402061853,
  'acousticness': 0.32432432432432434,
  'speechiness': 0.627906976744186,
  'popularity': 0.76,
  'cluster_val': 7,
  'distance': 1.109238641118842},
 {'unnamed: 0': 0.6326530612244898,
  'trackname': '7 rings',
  'artistname': 'ariana grande',
  'genre': 'dance pop',
  'beatsperminute': 0.5238095238095238,
  'energy': 0.0,
  'danceability': 0.8032786885245902,
  'loudnessdb': 0.0,
  'liveness': 0.07547169811320754,
  'valence': 0.27058823529411763,
  'length': 0.32989690721649484,
  'acousticness': 0.7837837837837838,
  'speechiness': 0.6976744186046512,
  'popularity': 0.76,
  'cluster_val': 7,
  'distance': 1.89858